# Pandas - Merge and Grouping

Some of the most interesting studies of data come from combining different data sources. These operations can involve anything from very straightforward concatenation of two different datasets. `Series` and DataFrames are built with this type of operation in mind, and `Pandas` includes functions and methods that its high-performance, in-memory concatenate, join and merge operations. 

* Simple concatenation of Series and DataFrames: `pd.concat()`
* Merge multiple dataframes using common columns/keys: `pd.merge()`

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('../Data/Height_Weight.csv')
df.head()

,Name,Gender,Height,Weight,Hometown
0,Ashley,F,155,140,Palo Alto
1,Robin,M,145,122,Fremont
2,Priyanka,M,152,131,Santa Clara
3,Youngchul,M,167,148,Cupertino
4,Aziz,F,161,139,San Francisco


## 1. Combining Datasets: Concat/Append

### 1.1. Simple Concatenation

Pandas has a function `pd.concat()`, which has a similar syntax to `np.concatenate` but contains a number of options that we'll discuss. `pd.concat()` can be used for a simple concatenation of Series or DataFrame objects, just as `np.concatenate()` can be used for simple concatenations of arrays

In [2]:
s1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
s2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([s1, s2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

In [3]:
# append row wise
temp = dict({'Name': ['Jay'], 'Height': [183.0], 'Weight': [165], 'Hometown': ['Thousand Oaks']})
pd.concat([df, pd.DataFrame(temp)], axis=0) # ignore_index to set correct index 

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,Gender,Height,Hometown,Name,Weight
0,F,155.0,Palo Alto,Ashley,140
1,M,145.0,Fremont,Robin,122
2,M,152.0,Santa Clara,Priyanka,131
3,M,167.0,Cupertino,Youngchul,148
4,F,161.0,San Francisco,Aziz,139
5,F,181.0,Hayward,Zoey,190
6,M,183.0,Los Angeles,Jay,180
7,F,172.0,Los Angeles,Frances,110
8,F,158.0,Santa Clara,Abby,120
9,F,162.0,San Francisco,Xia,110


In [4]:
# append column wise
temp = pd.Series(np.linspace(4,20,6))
pd.concat([df, temp], axis=1)

,Name,Gender,Height,Weight,Hometown,0
0,Ashley,F,155,140,Palo Alto,4.0
1,Robin,M,145,122,Fremont,7.2
2,Priyanka,M,152,131,Santa Clara,10.4
3,Youngchul,M,167,148,Cupertino,13.6
4,Aziz,F,161,139,San Francisco,16.8
5,Zoey,F,181,190,Hayward,20.0
6,Jay,M,183,180,Los Angeles,NaN
7,Frances,F,172,110,Los Angeles,NaN
8,Abby,F,158,120,Santa Clara,NaN
9,Xia,F,162,110,San Francisco,NaN


### 1.2. Duplicate indices

`pd.concat` is that Pandas concatenation preserves indices, even if the result will have duplicate indices! However, it might cause the data quality issues by duplicates.

In [5]:
d1 = dict({'Name': ['Jay', 'Zoey'], 'Height': [183.0, 161], 'Weight': [165, 139], 'Hometown': ['Thousand Oaks', 'San Francisco']})
d2 = dict({'Name': ['Ashley', 'Zoey'], 'Height': [155.0, 161.0], 'Weight': [140, 139], 'Hometown': ['Palo Alto', 'San Francisco']})

df1 = pd.DataFrame(d1, index=[0, 1])
df2 = pd.DataFrame(d2, index=[2, 1])

pd.concat([df1, df2])

,Name,Height,Weight,Hometown
0,Jay,183.0,165,Thousand Oaks
1,Zoey,161.0,139,San Francisco
2,Ashley,155.0,140,Palo Alto
1,Zoey,161.0,139,San Francisco


#### Catching the repeats as an error

If you'd like to simply verify that the indices in the result of `pd.concat` do not overlap, you can specify the verify_integrity flag. With this set to True, the concatenation will raise an exception if there are duplicate indices.

In [6]:
try:
    pd.concat([df1, df2], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Int64Index([1], dtype='int64')


#### Ignoring the index

Sometimes the index itself does not matter, and you would prefer it to simply be ignored. This option can be specified using the ignore_index flag. With this set to true, the concatenation will create a new integer index for the resulting Series.

In [7]:
pd.concat([df1, df2], ignore_index=True)

,Name,Height,Weight,Hometown
0,Jay,183.0,165,Thousand Oaks
1,Zoey,161.0,139,San Francisco
2,Ashley,155.0,140,Palo Alto
3,Zoey,161.0,139,San Francisco


#### Adding MultiIndex keys

Another option is to use the keys option to specify a label for the data sources; the result will be a hierarchically indexed series containing the data

In [8]:
pd.concat([df1, df2], keys=['df1', 'df2'])

Name  Height  Weight       Hometown
df1 0     Jay   183.0     165  Thousand Oaks
    1    Zoey   161.0     139  San Francisco
df2 2  Ashley   155.0     140      Palo Alto
    1    Zoey   161.0     139  San Francisco

### 1.3. Append DataFrame

Series and DataFrame objects have an append method that can accomplish the same thing in fewer keystrokes. Instead, it creates a new object with the combined data. It also is not a very efficient method because it involves creation of a new index and data buffer. **Thus, if you plan to do multiple append operations, it is generally better to build a list of DataFrames and pass them all at once to the `concat()` function.**

In [9]:
df1.append(df2)

,Name,Height,Weight,Hometown
0,Jay,183.0,165,Thousand Oaks
1,Zoey,161.0,139,San Francisco
2,Ashley,155.0,140,Palo Alto
1,Zoey,161.0,139,San Francisco


## 2. Combining Datasets: Join/Merge

### 2.1. Sinple Merge

One essential feature offered by Pandas is its high-performance, in-memory join and merge operations. Pandas implements several of these fundamental building-blocks in the `pd.merge()` function and the related `join()` method of Series and Dataframes. The `pd.merge()` function implements a number of types of joins: the one-to-one, many-to-one, and many-to-many joins.

However, often the column names will not match so nicely, so `pd.merge()` provides a variety of options for handling explicitly specify the name of the key column using the `on` keyword.

In [10]:
# Same join keys
df2 = pd.DataFrame({'Name': ['Ashley', 'Robin', 'Aziz', 'Zoey'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})

print(df2)
temp = pd.merge(df, df2, on='Name')
temp

     Name        group
0  Ashley   Accounting
1   Robin  Engineering
2    Aziz  Engineering
3    Zoey           HR


,Name,Gender,Height,Weight,Hometown,group
0,Ashley,F,155,140,Palo Alto,Accounting
1,Robin,M,145,122,Fremont,Engineering
2,Aziz,F,161,139,San Francisco,Engineering
3,Zoey,F,181,190,Hayward,HR


In [11]:
# Different join keys
df3 = pd.DataFrame({'depart': ['Accounting', 'Engineering', 'HR'],'supervisor': ['Carly', 'Guido', 'Steve']})
print(df3)
pd.merge(temp, df3, left_on='group', right_on='depart')

        depart supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve


,Name,Gender,Height,Weight,Hometown,group,depart,supervisor
0,Ashley,F,155,140,Palo Alto,Accounting,Accounting,Carly
1,Robin,M,145,122,Fremont,Engineering,Engineering,Guido
2,Aziz,F,161,139,San Francisco,Engineering,Engineering,Guido
3,Zoey,F,181,190,Hayward,HR,HR,Steve


Sometimes, rather than merging on a column, you would instead like to merge on an indx using `left_index` and `right_index` keywords. If you'd like to mix indices and columns, you can combine `index` with `on` to get the desired behavior.

In [12]:
# Join index & columns
df4 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']}).set_index('group')
print(df4)

pd.merge(temp, df4, left_on='group', right_index=True)

                   skills
group                    
Accounting           math
Accounting   spreadsheets
Engineering        coding
Engineering         linux
HR           spreadsheets
HR           organization


,Name,Gender,Height,Weight,Hometown,group,skills
0,Ashley,F,155,140,Palo Alto,Accounting,math
0,Ashley,F,155,140,Palo Alto,Accounting,spreadsheets
1,Robin,M,145,122,Fremont,Engineering,coding
1,Robin,M,145,122,Fremont,Engineering,linux
2,Aziz,F,161,139,San Francisco,Engineering,coding
2,Aziz,F,161,139,San Francisco,Engineering,linux
3,Zoey,F,181,190,Hayward,HR,spreadsheets
3,Zoey,F,181,190,Hayward,HR,organization


### 2.2. General Join Tables on Keys

By default, the result contains the intersection of the two sets of inputs; this is what is known as an inner join. We can specify this explicitly using the how keyword, which defaults to inner. Other options for the how keyword are 'outer', 'left', and 'right'. An outer join returns a join over the union of the input columns, and fills in all missing values with NAs.

In [13]:
# inner join
pd.merge(df, df2, on='Name', how='inner')

,Name,Gender,Height,Weight,Hometown,group
0,Ashley,F,155,140,Palo Alto,Accounting
1,Robin,M,145,122,Fremont,Engineering
2,Aziz,F,161,139,San Francisco,Engineering
3,Zoey,F,181,190,Hayward,HR


In [14]:
# left join
pd.merge(df, df2, on='Name', how='right')

,Name,Gender,Height,Weight,Hometown,group
0,Ashley,F,155,140,Palo Alto,Accounting
1,Robin,M,145,122,Fremont,Engineering
2,Aziz,F,161,139,San Francisco,Engineering
3,Zoey,F,181,190,Hayward,HR


In [15]:
# outer join
pd.merge(df, df2, on='Name', how='outer')

,Name,Gender,Height,Weight,Hometown,group
0,Ashley,F,155,140,Palo Alto,Accounting
1,Robin,M,145,122,Fremont,Engineering
2,Priyanka,M,152,131,Santa Clara,NaN
3,Youngchul,M,167,148,Cupertino,NaN
4,Aziz,F,161,139,San Francisco,Engineering
5,Zoey,F,181,190,Hayward,HR
6,Jay,M,183,180,Los Angeles,NaN
7,Frances,F,172,110,Los Angeles,NaN
8,Abby,F,158,120,Santa Clara,NaN
9,Xia,F,162,110,San Francisco,NaN


# Pandas - Grouping and Aggregation

An essential piece of analysis of large data is efficient summarization: computing aggregations in which a single number gives insight into the nature of a potentially large dataset. I will explore aggregations in Pandas, from simple operations and more sophisticated operations based on the concept of a groupby.

The following table summarizes some other built-in Pandas aggregations:

| Aggregation              | Description                     |
|--------------------------|---------------------------------|
| ``count()``              | Total number of items           |
| ``first()``, ``last()``  | First and last item             |
| ``mean()``, ``median()`` | Mean and median                 |
| ``min()``, ``max()``     | Minimum and maximum             |
| ``std()``, ``var()``     | Standard deviation and variance |
| ``mad()``                | Mean absolute deviation         |
| ``prod()``               | Product of all items            |
| ``sum()``                | Sum of all items                |

## 3. GroupBy: Split, Apply, Combine

Simple aggregations can give you a flavor of your dataset, but often we would prefer to aggregate conditionally on some label or index: this is implemented in the so-called `groupby` operation. Although the name "group by" comes from a command in the SQL database language, the `groupby` operation can be considered as split, apply, combine.

![title](../Data/Notebook_Images/Groupby.png)


This makes clear what the groupby accomplishes:

* The split step involves breaking up and grouping a DataFrame depending on the value of the specified key.
* The apply step involves computing some function, usually an aggregate, transformation, or filtering, within the individual groups.
* The combine step merges the results of these operations into an output array.

### 3.1. GroupBy object

The GroupBy object is a very flexible abstraction. The most basic split-apply-combine operation can be computed with the `groupby()` method of DataFrames, passing the name of the desired key column.

In [16]:
df.head()

,Name,Gender,Height,Weight,Hometown
0,Ashley,F,155,140,Palo Alto
1,Robin,M,145,122,Fremont
2,Priyanka,M,152,131,Santa Clara
3,Youngchul,M,167,148,Cupertino
4,Aziz,F,161,139,San Francisco


In [17]:
geo_gp = df.groupby('Hometown')
print('groupby object:', geo_gp, '\n')

groupby object: <pandas.core.groupby.generic.DataFrameGroupBy object at 0x0000016193EBA780> 



In [18]:
# iteration over the groups, returning groups as Series
for i,j in geo_gp:
    print('keys:', i, '\n', j, '\n')

keys: Cupertino 
         Name Gender  Height  Weight   Hometown
3  Youngchul      M     167     148  Cupertino 

keys: Fremont 
      Name Gender  Height  Weight Hometown
1   Robin      M     145     122  Fremont
10   Emma      F     165     120  Fremont 

keys: Hayward 
    Name Gender  Height  Weight Hometown
5  Zoey      F     181     190  Hayward 

keys: Los Angeles 
       Name Gender  Height  Weight     Hometown
6      Jay      M     183     180  Los Angeles
7  Frances      F     172     110  Los Angeles 

keys: Palo Alto 
       Name Gender  Height  Weight   Hometown
0   Ashley      F     155     140  Palo Alto
11   Terry      M     185     220  Palo Alto 

keys: San Francisco 
    Name Gender  Height  Weight       Hometown
4  Aziz      F     161     139  San Francisco
9   Xia      F     162     110  San Francisco 

keys: Santa Clara 
        Name Gender  Height  Weight     Hometown
2  Priyanka      M     152     131  Santa Clara
8      Abby      F     158     120  Santa Clara 

In [19]:
# select specific group in the groupby object
geo_gp.get_group('Los Angeles')

,Name,Gender,Height,Weight,Hometown
6,Jay,M,183,180,Los Angeles
7,Frances,F,172,110,Los Angeles


In [20]:
# customize columns name 
geo_gp.sum().add_prefix('sum_')

,sum_Height,sum_Weight
Hometown,,
Cupertino,167,148
Fremont,310,242
Hayward,181,190
Los Angeles,355,290
Palo Alto,340,360
San Francisco,323,249
Santa Clara,310,251


## 4. Aggregate, filter, transform, apply

In particular, GroupBy objects have `aggregate()`, `filter()`, `transform()`, and `apply()` methods that efficiently implement a variety of useful operations before combining the grouped data.

### 4.1. Aggregation

We're now familiar with GroupBy aggregations with sum(), median(), and the like, but the `aggregate()` method allows for even more flexibility. It can take a string, a function, or a list thereof, and compute all the aggregates at once.

In [21]:
df.groupby('Hometown').agg(['min', np.median, max])

Height             Weight            
                 min median  max    min median  max
Hometown                                           
Cupertino        167  167.0  167    148  148.0  148
Fremont          145  155.0  165    120  121.0  122
Hayward          181  181.0  181    190  190.0  190
Los Angeles      172  177.5  183    110  145.0  180
Palo Alto        155  170.0  185    140  180.0  220
San Francisco    161  161.5  162    110  124.5  139
Santa Clara      152  155.0  158    120  125.5  131

In [22]:
df.groupby(['Hometown']).agg({'Height': np.mean, 'Weight': np.min})

,Height,Weight
Hometown,,
Cupertino,167.0,148
Fremont,155.0,120
Hayward,181.0,190
Los Angeles,177.5,110
Palo Alto,170.0,140
San Francisco,161.5,110
Santa Clara,155.0,120


### 4.2.  Filtering

A filtering operation allows you to drop data based on the group properties like `having` in SQL

In [23]:
def filter_func(x):
    return x['Height'].mean() >= 170

df.groupby('Hometown').filter(filter_func)

,Name,Gender,Height,Weight,Hometown
0,Ashley,F,155,140,Palo Alto
5,Zoey,F,181,190,Hayward
6,Jay,M,183,180,Los Angeles
7,Frances,F,172,110,Los Angeles
11,Terry,M,185,220,Palo Alto


### 4.3. Transformation

`Transformation` can return some transformed version of the full data to recombine. For example, the output is the same shape as the input.

Following example is to center the data by subtracting the group-wise mean:

In [24]:
df.groupby('Hometown').transform(lambda x: x - x.mean())

,Height,Weight
0,-15.0,-40.0
1,-10.0,1.0
2,-3.0,5.5
3,0.0,0.0
4,-0.5,14.5
5,0.0,0.0
6,5.5,35.0
7,-5.5,-35.0
8,3.0,-5.5
9,0.5,-14.5


### 4.4. Apply

The `apply` method lets you apply an arbitrary function to the group results. The function should take a DataFrame, and return either a Pandas object (e.g., DataFrame, Series) or a scalar; the combine operation will be tailored to the type of output returned.

Following example is calculating the average BMI scores by city.

In [25]:
def BMI(x):
    # x is a DataFrame of group values
    x['avg_BMI'] = x['Weight'].mean()/2 / (x['Height'].mean()/100)**2
    return x

df.groupby('Hometown').apply(BMI)

,Name,Gender,Height,Weight,Hometown,avg_BMI
0,Ashley,F,155,140,Palo Alto,31.141869
1,Robin,M,145,122,Fremont,25.182102
2,Priyanka,M,152,131,Santa Clara,26.118626
3,Youngchul,M,167,148,Cupertino,26.533759
4,Aziz,F,161,139,San Francisco,23.866806
5,Zoey,F,181,190,Hayward,28.997894
6,Jay,M,183,180,Los Angeles,23.011307
7,Frances,F,172,110,Los Angeles,23.011307
8,Abby,F,158,120,Santa Clara,26.118626
9,Xia,F,162,110,San Francisco,23.866806


# Pandas - Pivot Tables

A `pivot table` is a similar operation that is commonly seen in spreadsheets and other programs that operate on tabular data. The pivot table takes simple column-wise data as input, and groups the entries into a two-dimensional table that provides a multidimensional summarization of the data, which helps to think of pivot tables as essentially a multidimensional version of `GroupBy` aggregation.

Following example is calculating average of member's height by gender and city levels

In [26]:
df.groupby(['Hometown', 'Gender']).Height.aggregate('mean').unstack()

Gender,F,M
Hometown,,
Cupertino,NaN,167.0
Fremont,165.0,145.0
Hayward,181.0,NaN
Los Angeles,172.0,183.0
Palo Alto,155.0,185.0
San Francisco,161.5,NaN
Santa Clara,158.0,152.0


This two-dimensional GroupBy is common enough that Pandas includes a convenience routine, pivot_table, which succinctly handles this type of multi-dimensional aggregation.

In [27]:
df.pivot_table('Height', index='Hometown', columns='Gender', aggfunc='mean')

Gender,F,M
Hometown,,
Cupertino,NaN,167.0
Fremont,165.0,145.0
Hayward,181.0,NaN
Los Angeles,172.0,183.0
Palo Alto,155.0,185.0
San Francisco,161.5,NaN
Santa Clara,158.0,152.0


At times it's useful to compute totals along each grouping, using `margins` keyword.

In [28]:
df.pivot_table('Height', index='Hometown', columns='Gender', aggfunc='mean', margins=True, margins_name='Overall_avg')

Gender,F,M,Overall_avg
Hometown,,,
Cupertino,NaN,167.0,167.0
Fremont,165.000000,145.0,155.0
Hayward,181.000000,NaN,181.0
Los Angeles,172.000000,183.0,177.5
Palo Alto,155.000000,185.0,170.0
San Francisco,161.500000,NaN,161.5
Santa Clara,158.000000,152.0,155.0
Overall_avg,164.857143,166.4,165.5


Additionally, it can be specified as a dictionary mapping a column to any of the above desired options

In [29]:
df.pivot_table(index='Hometown', columns='Gender', aggfunc={'Height':'mean', 'Weight':'median'})

Height        Weight       
Gender             F      M      F      M
Hometown                                 
Cupertino        NaN  167.0    NaN  148.0
Fremont        165.0  145.0  120.0  122.0
Hayward        181.0    NaN  190.0    NaN
Los Angeles    172.0  183.0  110.0  180.0
Palo Alto      155.0  185.0  140.0  220.0
San Francisco  161.5    NaN  124.5    NaN
Santa Clara    158.0  152.0  120.0  131.0